# Modules

In [1]:
from pdathome.constants import global_constants as gc
from pdathome.classification import train_test_filtering_gait, store_filtering_gait

# Process data

In [ ]:
l_classifiers = [gc.classifiers.LOGISTIC_REGRESSION]

In [3]:
for subject in gc.participant_ids.L_PD_IDS:
    train_test_filtering_gait(subject, l_classifiers=['logreg'])

store_filtering_gait(l_classifiers=['logreg'])